In [35]:
%load_ext autoreload
%autoreload 2

In [36]:
import os 
funcs = os.listdir('functions')

for func in funcs:
    if func.startswith('__') or func == 'import_all_functions.py':
        continue
    func = func.replace('.py','')
    print(f'{func} imported')
    exec(f'from functions.{func} import *')

import_to_df imported
add_had_remod_flag imported
get_kaggle_data imported
add_years_since imported
add_year_remod_pre1950_flag imported
quality_to_number imported
add_dummies_multiple_columns imported
add_value_dummies_multiple_columns imported
process_data imported
sum_onehots imported
add_zero_flag imported


In [138]:
import pandas as pd 
import pandas_profiling as pdp
import numpy as np
import pprint as pp
import seaborn as sns
from matplotlib import pyplot as plt
from catboost import Pool, CatBoostRegressor

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_val_score


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [39]:
files = get_kaggle_data(
    competition="house-prices-advanced-regression-techniques"
    )

files downloaded for house-prices-advanced-regression-techniques competition
   test.csv
   data_description.txt
   train.csv
   sample_submission.csv
located in /competition_data/


In [40]:
input_data = import_to_df(files)

What files did we get?

In [41]:
input_data.keys()

dict_keys(['test', 'train', 'sample_submission'])

In [42]:
input_train_df = input_data['train']
input_test_df = input_data['test']

In [115]:
train_df = process_data(input_train_df)
target = 'SalePrice'
id_row = 'Id'
train_predictors = [
    c for c in train_df.columns 
    if target not in c
    if id_row not in c
]

In [116]:
test_df = process_data(input_test_df)
test_predictors = [
    c for c in test_df.columns 
    if target not in c
    if id_row not in c
]

Only keep columns in both dfs. Needed in case one hots generate unique columns

In [117]:
cols_to_use = list(set(train_predictors).intersection(test_predictors))

In [118]:
len(cols_to_use), len(train_predictors)

(299, 311)

Removed 1 column

In [119]:
X_train = train_df[cols_to_use]
y_train = train_df[target]

X_test = test_df[cols_to_use]
y_test_labels = test_df[id_row]

In [93]:
# initialize Pool
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test) 

param_dist = {
    'depth': [6,8,10],
    'learning_rate' : [0.01, 0.05, 0.1],
    'iterations'    : [30, 50, 100],
    'l2_leaf_reg': [1, 3, 5, 7, 9]
}
#Instantiate RandomSearchCV object
cbr = CatBoostRegressor()

random_grid = RandomizedSearchCV(cbr, param_dist, cv =5)

#Fit the model
random_grid.fit(X_train, y_train, verbose=2)

# Print the tuned parameters and score
print(random_grid.best_params_)
print(random_grid.best_score_)
# specify the training parameters 
# model = CatBoostRegressor(
#     iterations=5, 
#     depth=2, 
#     learning_rate=1, 
#     loss_function='RMSE'
# )
#train the model
# model.fit(train_pool)
# make the prediction using the resulting model
# preds = model.predict(test_pool)

1	total: 1.06s	remaining: 335ms
38:	learn: 34110.5693836	total: 1.09s	remaining: 308ms
39:	learn: 33729.1323280	total: 1.12s	remaining: 279ms
40:	learn: 33282.1824698	total: 1.14s	remaining: 251ms
41:	learn: 32868.8721144	total: 1.17s	remaining: 222ms
42:	learn: 32421.3925510	total: 1.2s	remaining: 195ms
43:	learn: 32011.2473726	total: 1.22s	remaining: 167ms
44:	learn: 31583.6707582	total: 1.25s	remaining: 139ms
45:	learn: 31265.0101817	total: 1.27s	remaining: 111ms
46:	learn: 30991.0426080	total: 1.3s	remaining: 83ms
47:	learn: 30588.0991227	total: 1.32s	remaining: 55.1ms
48:	learn: 30224.3412273	total: 1.34s	remaining: 27.5ms
49:	learn: 29967.6079067	total: 1.36s	remaining: 0us
0:	learn: 79005.8355710	total: 17.7ms	remaining: 866ms
1:	learn: 76557.5544958	total: 39.2ms	remaining: 942ms
2:	learn: 74555.6492547	total: 65.5ms	remaining: 1.03s
3:	learn: 72519.3838503	total: 92.1ms	remaining: 1.06s
4:	learn: 70536.9026578	total: 119ms	remaining: 1.07s
5:	learn: 68617.8677199	total: 150ms	

Use these params to start grid search

In [98]:
param_grid = {
    'learning_rate': [0.05, 0.1],
    'iterations': [80, 100, 150,], 
    'depth': [4,8,10],
    'l2_leaf_reg': [5, 7,],
    'random_strength': [1,2]
}

grid_search = GridSearchCV(estimator = cbr, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits
0:	learn: 75041.9249194	total: 5.28ms	remaining: 787ms
1:	learn: 70608.9577935	total: 12.1ms	remaining: 896ms
2:	learn: 66771.8136501	total: 16.4ms	remaining: 805ms
3:	learn: 63069.0289300	total: 20.8ms	remaining: 759ms
4:	learn: 59650.5016395	total: 26.5ms	remaining: 767ms
5:	learn: 56738.5533641	total: 31.6ms	remaining: 759ms
6:	learn: 54529.2532115	total: 36.8ms	remaining: 751ms
7:	learn: 52290.3892366	total: 42.6ms	remaining: 757ms
8:	learn: 50019.7825485	total: 48.2ms	remaining: 755ms
9:	learn: 48122.7219634	total: 56.2ms	remaining: 787ms
10:	learn: 46099.6746810	total: 62.6ms	remaining: 792ms
11:	learn: 44454.5407451	total: 70.9ms	remaining: 816ms
12:	learn: 43031.5694179	total: 76.8ms	remaining: 810ms
13:	learn: 41630.6898660	total: 82.2ms	remaining: 799ms
14:	learn: 40496.7922046	total: 88.6ms	remaining: 797ms
15:	learn: 39378.3427527	total: 93.9ms	remaining: 786ms
16:	learn: 38344.9145233	total: 99.6ms	remaining: 78

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostRegressor object at 0x7fb9d23a5340>,
             n_jobs=-1,
             param_grid={'depth': [4, 8, 10], 'iterations': [80, 100, 150],
                         'l2_leaf_reg': [5, 7], 'learning_rate': [0.05, 0.1],
                         'random_strength': [1, 2]},
             verbose=1)

In [99]:
print(grid_search.best_params_)
print(grid_search.best_score_)

{'depth': 4, 'iterations': 150, 'l2_leaf_reg': 5, 'learning_rate': 0.1, 'random_strength': 1}
0.8819582287128055


## Random Forest

First use random search again

In [120]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 40]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 5, 10, 50]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

In [121]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4, 5, 10,
                                                             50],
                                        'min_samples_split': [2, 5, 10, 40],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [123]:
display(rf_random.best_params_)
display(rf_random.best_score_)

{'n_estimators': 1200,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 110,
 'bootstrap': False}

0.8644008815472926

Use these parameters to create full grid search

In [125]:
rf_param_grid = {
    'bootstrap': [False],
    'max_depth': [ 100, 110, 120],
    'max_features': ['sqrt'],
    'min_samples_leaf': [ 1, 10, 50],
    'min_samples_split': [ 2, 6, 12, 18],
    'n_estimators': [ 1000, 1200, 1400]
}

rf_grid_search = GridSearchCV(estimator = rf, param_grid = rf_param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)
rf_grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [False], 'max_depth': [100, 110, 120],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 10, 50],
                         'min_samples_split': [2, 6, 12, 18],
                         'n_estimators': [1000, 1200, 1400]},
             verbose=2)

In [126]:
display(rf_grid_search.best_params_)
display(rf_grid_search.best_score_)

{'bootstrap': False,
 'max_depth': 110,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1200}

0.8654819443903559

## Elastic Search
Try a linear model for rounding off ensembling. Apparently this is good for many features not enough obs

In [129]:
en_grid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0],
    'l1_ratio': np.arange(0, 1, 0.01)
}
en = ElasticNet()
en_grid_search = GridSearchCV(estimator = en, param_grid = en_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)

In [130]:
display(rf_grid_search.best_params_)
display(rf_grid_search.best_score_)

{'bootstrap': False,
 'max_depth': 110,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1200}

0.8654819443903559

Ensemble them all together

In [139]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)

ensemble = VotingRegressor(
    estimators=[
        ('ElasticNet', en_grid_search),
        ('RF Grid', rf_grid_search),
        ('RF Random', rf_random),
        ('Grad Boost', grid_search)
    ])
scores_best_ens = cross_val_score(ensemble, X_train, y_train, cv=cv, n_jobs=-1)
scores_best_ens

TypeError: __init__() got an unexpected keyword argument 'voting'

# make preds and submit

In [100]:
y_pred = grid_search.predict(X_test)

submission_df = pd.DataFrame()
submission_df[id_row] = y_test_labels
submission_df[target] = y_pred

In [101]:
submission_df.head()

,Id,SalePrice
0,1461,128494.894968
1,1462,163919.356654
2,1463,175741.863058
3,1464,185275.158473
4,1465,190155.918799


In [102]:
input_data['sample_submission'].head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [104]:
submission_df.to_csv('submission2.csv', index=False)

In [105]:
!kaggle competitions submit -f 'submission2.csv' -m 'grid search catboost' house-prices-advanced-regression-techniques

100%|██████████████████████████████████████| 33.6k/33.6k [00:02<00:00, 12.2kB/s]
Successfully submitted to House Prices - Advanced Regression Techniques

In [108]:
!kaggle competitions submissions -c house-prices-advanced-regression-techniques

fileName         date                 description                           status    publicScore  privateScore  
---------------  -------------------  ------------------------------------  --------  -----------  ------------  
submission2.csv  2021-05-04 07:17:54  grid search catboost                  complete  0.13561      None          
submission1.csv  2021-05-03 11:30:27  first attempt random hypers catboost  complete  0.15473      None          
